# Retrieval Augmented Generation

Table of Contents
1. Idea
2. Naive Implementation
3. Graph Implementation

In [1]:
from language_models.proxy_client import BTPProxyClient
from language_models.agents.react import ReActAgent
from language_models.agents.chain import AgentChain
from language_models.tools.tool import Tool
from language_models.models.llm import OpenAILanguageModel
from language_models.models.embedding import SentenceTransformerEmbeddingModel
from language_models.retrievers import BasicRetriever, ContextualCompressionRetriever
from language_models.retrievers.utils import split_documents
from language_models.vector_stores import FAISSVectorStore, DistanceMetric
from language_models.settings import settings
from langchain_community.document_loaders.recursive_url_loader import RecursiveUrlLoader
from utils import load_docs_from_json, save_docs_to_json
from pydantic import BaseModel, Field
from numpy import dot
from numpy.linalg import norm
from bs4 import BeautifulSoup as Soup

13/05/24 11:57:29 INFO Load pretrained SentenceTransformer: sentence-transformers/all-MiniLM-L6-v2
13/05/24 11:57:36 INFO Use pytorch device_name: mps
13/05/24 11:57:37 INFO Loading faiss.
13/05/24 11:57:37 INFO Successfully loaded faiss.


In [2]:
proxy_client = BTPProxyClient(
    client_id=settings.CLIENT_ID,
    client_secret=settings.CLIENT_SECRET,
    auth_url=settings.AUTH_URL,
    api_base=settings.API_BASE,
)

In [3]:
try:
    documents = load_docs_from_json('data.json')
except:
    documents = []
    urls = [
        'https://www.apple.com/iphone/',
        'https://www.apple.com/ipad/',
        'https://www.apple.com/watch/',
        'https://www.apple.com/airpods/',
        'https://www.apple.com/mac/',
        'https://store.google.com/us/category/phones?hl=en-US',
        'https://store.google.com/us/category/watches_trackers?hl=en-US',
        'https://store.google.com/us/product/pixel_tablet?hl=en-US',
        'https://store.google.com/us/category/earbuds?hl=en-US',
        'https://www.samsung.com/us/smartphones/',
        'https://www.samsung.com/us/watches/',
        'https://www.samsung.com/us/tablets/',
        'https://www.samsung.com/us/mobile-audio/',
        'https://www.samsung.com/us/galaxybooks/',
    ]
    for url in urls:
        loader = RecursiveUrlLoader(url=url, max_depth=5, extractor=lambda x: Soup(x, "html.parser").text)
        docs = loader.load()
        documents += docs
    save_docs_to_json(documents, 'data.json')

In [4]:
documents = split_documents(documents, separators=["\n\n", "\n", " ", ""], chunk_size=4000, chunk_overlap=200)

## Idea

In [5]:
llm = OpenAILanguageModel(
    proxy_client=proxy_client,
    model="gpt-4",
    max_tokens=256,
    temperature=0.0,
)

In [6]:
class Output(BaseModel):
    content: str = Field(description="The final answer.")

agent = ReActAgent.create(
    llm=llm,
    system_prompt="",
    task_prompt="{question}",
    task_prompt_variables=["question"],
    output_format=Output,
    iterations=3,
)

In [7]:
response = agent.invoke({"question": "Tell me about the iPhone 15 pro and pro max."})

13/05/24 11:59:09 INFO Prompt:
Tell me about the iPhone 15 pro and pro max.
13/05/24 11:59:18 INFO Raw response:
{
  "thought": "The user is asking about the iPhone 15 Pro and Pro Max, but as of my last update in 2021, Apple has only released up to the iPhone 13 Pro and Pro Max. I should inform the user about this.",
  "tool": "Final Answer",
  "tool_input": {"content": "As of my last update in 2021, Apple has only released up to the iPhone 13 Pro and Pro Max. The iPhone 15 Pro and Pro Max do not exist yet. Please check Apple's official website or contact Apple directly for the most current information."}
}
13/05/24 11:59:18 INFO Thought:
The user is asking about the iPhone 15 Pro and Pro Max, but as of my last update in 2021, Apple has only released up to the iPhone 13 Pro and Pro Max. I should inform the user about this.
13/05/24 11:59:18 INFO Final answer:
{'content': "As of my last update in 2021, Apple has only released up to the iPhone 13 Pro and Pro Max. The iPhone 15 Pro and Pr

In [8]:
print(response.final_answer["content"])

As of my last update in 2021, Apple has only released up to the iPhone 13 Pro and Pro Max. The iPhone 15 Pro and Pro Max do not exist yet. Please check Apple's official website or contact Apple directly for the most current information.


In [9]:
question = """Tell me about the iPhone 15 pro and pro max.

Use this context to answer the question:
- iPhone 15 Pro and Pro Max are the latest models.
- Available in Natural, Blue, White, and Black titanium.
- A17 Pro chip with 6-core GPU for fast performance.
- Pro camera: 48MP Main, 3x or 5x Telephoto, Ultra Wide.
- Up to 29 hours of video playback.
- ProMotion and Always-On display.
- Titanium back with textured matte glass.
- USB-C for 20x faster transfers.
- Emergency SOS, Crash Detection, Roadside Assistance via satellite."""

response = agent.invoke({"question": question})

13/05/24 11:59:18 INFO Prompt:
Tell me about the iPhone 15 pro and pro max.

Use this context to answer the question:
- iPhone 15 Pro and Pro Max are the latest models.
- Available in Natural, Blue, White, and Black titanium.
- A17 Pro chip with 6-core GPU for fast performance.
- Pro camera: 48MP Main, 3x or 5x Telephoto, Ultra Wide.
- Up to 29 hours of video playback.
- ProMotion and Always-On display.
- Titanium back with textured matte glass.
- USB-C for 20x faster transfers.
- Emergency SOS, Crash Detection, Roadside Assistance via satellite.
13/05/24 11:59:27 INFO Raw response:
{
  "thought": "I now know what to respond",
  "tool": "Final Answer",
  "tool_input": {
    "content": "The iPhone 15 Pro and Pro Max are the latest models from Apple. They are available in Natural, Blue, White, and Black titanium. They are powered by the A17 Pro chip with a 6-core GPU for fast performance. The Pro camera system includes a 48MP Main camera, a 3x or 5x Telephoto camera, and an Ultra Wide ca

In [10]:
print(response.final_answer["content"])

The iPhone 15 Pro and Pro Max are the latest models from Apple. They are available in Natural, Blue, White, and Black titanium. They are powered by the A17 Pro chip with a 6-core GPU for fast performance. The Pro camera system includes a 48MP Main camera, a 3x or 5x Telephoto camera, and an Ultra Wide camera. They offer up to 29 hours of video playback. The display features ProMotion and Always-On technology. The back is made of titanium with textured matte glass. They use USB-C for 20x faster transfers. They also have features like Emergency SOS, Crash Detection, and Roadside Assistance via satellite.


In [11]:
embedding_model = SentenceTransformerEmbeddingModel(model="all-MiniLM-L6-v2")

In [12]:
query1 = "Tell me about the iPhone 15 pro and pro max."
embedding1 = embedding_model.embed_query(query1)

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

In [13]:
print(embedding1)

[-0.04976186528801918, 0.024698281660676003, 0.024233346804976463, -0.08842168003320694, 0.0374833419919014, -0.01828034222126007, 0.004888656549155712, 0.11331886053085327, 0.016784582287073135, 0.08297941088676453, 0.10016509890556335, 0.05813554301857948, 0.0017647211207076907, 0.04127701371908188, 0.010860818438231945, 0.03729262202978134, 0.1335870325565338, -0.08417602628469467, -0.05983923748135567, -0.10192283987998962, -0.01667306385934353, -0.07314975559711456, 0.06727840006351471, 0.049850501120090485, 0.04839256778359413, 0.05804368853569031, -0.06915360689163208, -0.009908149018883705, -0.03353537619113922, -0.0551525242626667, 0.02574399672448635, -0.02915993705391884, 0.07582292705774307, -0.013736454769968987, -0.013520478270947933, -0.08669689297676086, 0.0033832842018455267, -0.023326121270656586, 0.008765621110796928, -0.06883351504802704, -0.03609778359532356, -0.0032673256937414408, -0.056928034871816635, 0.052639756351709366, 0.05331815406680107, -0.01249685883522

In [14]:
query2 = """- iPhone 15 Pro and Pro Max are the latest models.
- Available in Natural, Blue, White, and Black titanium.
- A17 Pro chip with 6-core GPU for fast performance.
- Pro camera: 48MP Main, 3x or 5x Telephoto, Ultra Wide.
- Up to 29 hours of video playback.
- ProMotion and Always-On display.
- Titanium back with textured matte glass.
- USB-C for 20x faster transfers.
- Emergency SOS, Crash Detection, Roadside Assistance via satellite."""
embedding2 = embedding_model.embed_query(query2)

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

In [15]:
cosine_similarity = dot(embedding1, embedding2) / (norm(embedding1) * norm(embedding2))
print(f"Cosine similarity: {cosine_similarity}")

Cosine similarity: 0.738208049156622


## Naive Implementation

In [16]:
vector_store = FAISSVectorStore.from_documents(
    documents=documents,
    embedding_model=embedding_model,
    distance_metric=DistanceMetric.COSINE_SIMILARITY,
)

Batches:   0%|          | 0/35 [00:00<?, ?it/s]

In [17]:
tool_name = "Search"
tool_description = "Use this tool to search the latest products by Apple, Google, and Samsung."

class Search(BaseModel):
    user_text: str = Field(description="The user question/prompt/text.")
    fetch_k: int = Field(5, description="The maximum number of documents to return.")

In [18]:
basic_retriever = BasicRetriever(
    vector_store=vector_store,
    score_threshold=0.0
)

basic_retriever_tool = Tool(
    func=basic_retriever.get_relevant_documents,
    name=tool_name,
    description=tool_description,
    args_schema=Search,
)

In [19]:
class Output(BaseModel):
    content: str = Field(description="The final answer.")

basic_retriever_agent = ReActAgent.create(
    llm=llm,
    system_prompt="",
    task_prompt="{question}",
    task_prompt_variables=["question"],
    output_format=Output,
    tools=[basic_retriever_tool],
    iterations=3,
)

In [20]:
response = basic_retriever_agent.invoke({"question": "Tell me about the iPhone 15 pro and pro max."})

13/05/24 11:59:32 INFO Prompt:
Tell me about the iPhone 15 pro and pro max.
13/05/24 11:59:37 INFO Raw response:
{
  "thought": "The user is asking about the iPhone 15 pro and pro max. However, as of now, these models do not exist. The latest models from Apple are the iPhone 13 Pro and Pro Max. I should use the Search tool to find the latest information about these models.",
  "tool": "Search",
  "tool_input": {"user_text": "iPhone 13 Pro and Pro Max", "fetch_k": 5}
}
13/05/24 11:59:37 INFO Thought:
The user is asking about the iPhone 15 pro and pro max. However, as of now, these models do not exist. The latest models from Apple are the iPhone 13 Pro and Pro Max. I should use the Search tool to find the latest information about these models.
13/05/24 11:59:37 INFO Tool:
Search
13/05/24 11:59:37 INFO Tool input:
{'user_text': 'iPhone 13 Pro and Pro Max', 'fetch_k': 5}


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

13/05/24 11:59:37 INFO Tool response:
iPhone - Compare Models - Apple



















AppleStoreMaciPadiPhoneWatchVisionAirPodsTV & HomeEntertainmentAccessoriesSupport


0+










Get credit toward iPhone 15 or iPhone 15 Pro when you trade in an eligible smartphone.Get credit toward iPhone 15 or iPhone 15 Pro when you trade in an eligible smartphone.** Shop iPhone







Compare iPhone models

Shop iPhone


Get help choosing.
Chat with a Specialist




Watch a guided tour of iPhone 15 and iPhone 15 Pro 







iPhone 15 Pro


iPhone 15 Pro Max


iPhone 15


iPhone 15 Plus


iPhone 14 Pro


iPhone 14 Pro Max


iPhone 14


iPhone 14 Plus


iPhone SE (3rd generation)


iPhone 13 Pro


iPhone 13 Pro Max


iPhone 13 mini


iPhone 13


iPhone 12 Pro


iPhone 12 Pro Max


iPhone 12 mini


iPhone 12


iPhone SE (2nd generation)


iPhone 11 Pro


iPhone 11 Pro Max


iPhone 11


iPhone XS


iPhone XS Max


iPhone XR


iPhone X


iPhone 8 Plus


iPhone 8


iPhone 7 Plus


iPhone 7


iPhone 

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

13/05/24 11:59:44 INFO Tool response:
iPhone - Compare Models - Apple



















AppleStoreMaciPadiPhoneWatchVisionAirPodsTV & HomeEntertainmentAccessoriesSupport


0+










Get credit toward iPhone 15 or iPhone 15 Pro when you trade in an eligible smartphone.Get credit toward iPhone 15 or iPhone 15 Pro when you trade in an eligible smartphone.** Shop iPhone







Compare iPhone models

Shop iPhone


Get help choosing.
Chat with a Specialist




Watch a guided tour of iPhone 15 and iPhone 15 Pro 







iPhone 15 Pro


iPhone 15 Pro Max


iPhone 15


iPhone 15 Plus


iPhone 14 Pro


iPhone 14 Pro Max


iPhone 14


iPhone 14 Plus


iPhone SE (3rd generation)


iPhone 13 Pro


iPhone 13 Pro Max


iPhone 13 mini


iPhone 13


iPhone 12 Pro


iPhone 12 Pro Max


iPhone 12 mini


iPhone 12


iPhone SE (2nd generation)


iPhone 11 Pro


iPhone 11 Pro Max


iPhone 11


iPhone XS


iPhone XS Max


iPhone XR


iPhone X


iPhone 8 Plus


iPhone 8


iPhone 7 Plus


iPhone 7


iPhone 

In [21]:
print(response.final_answer["content"])

Currently, the latest iPhone models are the iPhone 14 and iPhone 14 Pro. The iPhone 15 series has not been released yet. Please keep an eye on Apple's official announcements for the latest information.


In [22]:
contextual_compression_retriever = ContextualCompressionRetriever(
    llm=llm,
    vector_store=vector_store,
    score_threshold=0.0
)

contextual_compression_retriever_tool = Tool(
    func=contextual_compression_retriever.get_relevant_documents,
    name=tool_name,
    description=tool_description,
    args_schema=Search,
)

In [23]:
class Output(BaseModel):
    content: str = Field(description="The final answer.")

contextual_compression_retriever_agent = ReActAgent.create(
    llm=llm,
    system_prompt="",
    task_prompt="{question}",
    task_prompt_variables=["question"],
    output_format=Output,
    tools=[contextual_compression_retriever_tool],
    iterations=3,
)

In [24]:
response = basic_retriever_agent.invoke({"question": "Tell me about the iPhone 15 pro and pro max."})

13/05/24 11:59:50 INFO Prompt:
Tell me about the iPhone 15 pro and pro max.
13/05/24 11:59:56 INFO Raw response:
{
  "thought": "The user wants to know about the iPhone 15 Pro and Pro Max. I will use the Search tool to find the latest information about these models.",
  "tool": "Search",
  "tool_input": {
    "user_text": "iPhone 15 Pro and Pro Max",
    "fetch_k": 5
  }
}
13/05/24 11:59:56 INFO Thought:
The user wants to know about the iPhone 15 Pro and Pro Max. I will use the Search tool to find the latest information about these models.
13/05/24 11:59:56 INFO Tool:
Search
13/05/24 11:59:56 INFO Tool input:
{'user_text': 'iPhone 15 Pro and Pro Max', 'fetch_k': 5}


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

13/05/24 11:59:56 INFO Tool response:
AT&T iPhone 15 Special Deal: Monthly price reflects net monthly payment, after application of AT&T trade‑in credit applied over 36 months with purchase of an iPhone 15 Pro, iPhone 15 Pro Max, iPhone 15, or iPhone 15 Plus and trade‑in of eligible smartphone. Receive credit with purchase of an iPhone 15 Pro or iPhone 15 Pro Max of either $1000, $700, or $350 (based upon the model and condition of your trade‑in smartphone). Receive credit with purchase of an iPhone 15 or iPhone 15 Plus of either $700 or $350 (based upon the model and condition of your trade‑in smartphone). Max bill credits will not exceed the cost of the device. Requires upgrade of an existing line or activation of a new line and purchase of a new iPhone 15 Pro, iPhone 15 Pro Max, iPhone 15, or iPhone 15 Plus on qualifying 36 month 0% APR installment plan, subject to carrier credit qualification. AT&T Installment Plan with Next Up is not eligible for this promotion. $0 down for well q

In [25]:
print(response.final_answer["content"])

I'm sorry for the confusion, but as of now, the latest iPhone models are the iPhone 13 Pro and Pro Max. The iPhone 15 Pro and Pro Max do not exist yet. If you have any questions about the iPhone 13 Pro and Pro Max, feel free to ask!


## Graph Implementation